In [2]:
# Gerekli kütüphaneler
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from skimage.feature import hog
import matplotlib.pyplot as plt


In [3]:
#Verileri alıyorum
data_directory = "Data"  # senin kullandığın değişken adı

# MAC otomatik bir dosya oluşturuyor klasörde .DS_Store isimli bu yüzden böyle alıyorum
siniflar = sorted(
    [d for d in os.listdir(data_directory)
     if os.path.isdir(os.path.join(data_directory, d))]
)




In [4]:
print("Sınıflar:", siniflar)

Sınıflar: ['Angry', 'Happy', 'Neutral', 'Sad', 'Surprise']


In [5]:
goruntuler = []
etiketler = []


for etiket, cls in enumerate(siniflar):
    klasor = os.path.join(data_directory, cls)
    dosyalar = os.listdir(klasor)
    for dosya in dosyalar:
        dosya_yolu = os.path.join(klasor, dosya)
        img = cv2.imread(dosya_yolu)
        if img is None:
            continue
        #Verilerim RGB veriler olduğundan griye çeviriyorum 
        gri = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        yeniden_boyutlu = cv2.resize(gri, (64, 64))
        normalize = yeniden_boyutlu / 255.0
        goruntuler.append(normalize)
        etiketler.append(etiket)

goruntuler = np.array(goruntuler)
etiketler = np.array(etiketler)

In [6]:
print("Toplam görüntü sayısı:", len(goruntuler))

Toplam görüntü sayısı: 14248


In [7]:
#HOG Öznitelik Çıkarımı
hog_ozellikleri = []

for idx, img in enumerate(goruntuler):
    ozellik = hog(
        img,
        orientations=9,
        pixels_per_cell=(8, 8),
        cells_per_block=(2, 2),
        block_norm='L2-Hys'
    )
    hog_ozellikleri.append(ozellik)
    
    if idx % 1000 == 0:
        print(f"{idx} görüntü işlendi.")

hog_ozellikleri = np.array(hog_ozellikleri)

print("HOG matris boyutu:", hog_ozellikleri.shape)

0 görüntü işlendi.
1000 görüntü işlendi.
2000 görüntü işlendi.
3000 görüntü işlendi.
4000 görüntü işlendi.
5000 görüntü işlendi.
6000 görüntü işlendi.
7000 görüntü işlendi.
8000 görüntü işlendi.
9000 görüntü işlendi.
10000 görüntü işlendi.
11000 görüntü işlendi.
12000 görüntü işlendi.
13000 görüntü işlendi.
14000 görüntü işlendi.
HOG matris boyutu: (14248, 1764)


In [8]:
print("Örnek HOG vektörü (ilk görüntü):")
print(hog_ozellikleri[0])


Örnek HOG vektörü (ilk görüntü):
[0.21285212 0.32703705 0.23865573 ... 0.02189057 0.13600681 0.01468464]


In [9]:
# Veriyi eğitim ve test olarak bölüyorum
X_train, X_test, y_train, y_test = train_test_split(
    hog_ozellikleri, etiketler,
    test_size=0.2,
    random_state=42,
    stratify=etiketler
)

print("Eğitim seti boyutu:", X_train.shape)
print("Test seti boyutu:", X_test.shape)

Eğitim seti boyutu: (11398, 1764)
Test seti boyutu: (2850, 1764)


In [9]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import time

# Proje kapsamı gereği farklı parametrelerini deniyorum
modeller = [
    {"C": 1, "gamma": 'scale'},
    {"C": 10, "gamma": 'scale'},
    {"C": 1, "gamma": 0.01},
    {"C": 10, "gamma": 0.01},
    {"C": 100, "gamma": 'scale'},
    {"C": 100, "gamma": 0.01},
]

for idx, parametreler in enumerate(modeller, 1):
    print(f"\n--- SVM Model {idx}: C={parametreler['C']}, gamma={parametreler['gamma']} ---")
    
    # Eğitim süresi ölç
    egitim_basla = time.time()
    svm_model = SVC(
        kernel='rbf',
        C=parametreler["C"],
        gamma=parametreler["gamma"]
    )
    svm_model.fit(X_train, y_train)
    egitim_bitis = time.time()
    
    # Test süresi ölç
    test_basla = time.time()
    y_pred = svm_model.predict(X_test)
    test_bitis = time.time()
    
    # Doğruluk
    dogruluk = accuracy_score(y_test, y_pred)
    print("Doğruluk:", dogruluk)
    print(classification_report(y_test, y_pred))
    
    # Süreleri yazdır
    egitim_sure = egitim_bitis - egitim_basla
    test_sure = test_bitis - test_basla
    print(f"Eğitim süresi: {egitim_sure:.2f} saniye")
    print(f"Test süresi: {test_sure:.2f} saniye")


--- SVM Model 1: C=1, gamma=scale ---
Doğruluk: 0.647719298245614
              precision    recall  f1-score   support

           0       0.58      0.41      0.48       263
           1       0.76      0.83      0.79       748
           2       0.61      0.67      0.64       805
           3       0.59      0.57      0.58       787
           4       0.65      0.54      0.59       247

    accuracy                           0.65      2850
   macro avg       0.64      0.60      0.62      2850
weighted avg       0.64      0.65      0.64      2850

Eğitim süresi: 82.47 saniye
Test süresi: 16.36 saniye

--- SVM Model 2: C=10, gamma=scale ---
Doğruluk: 0.6540350877192982
              precision    recall  f1-score   support

           0       0.55      0.49      0.52       263
           1       0.77      0.83      0.80       748
           2       0.62      0.64      0.63       805
           3       0.60      0.59      0.59       787
           4       0.66      0.55      0.60       

In [10]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
import time

# Farklı MLP parametrelerini proje gereksinimleri doğrultusunda burada deniyorum
modeller = [
    {"hidden_layer_sizes": (100,), "learning_rate_init": 0.001, "alpha":0.0001},
    {"hidden_layer_sizes": (200, 100), "learning_rate_init": 0.001, "alpha":0.0001},
    {"hidden_layer_sizes": (300, 200, 100), "learning_rate_init": 0.001, "alpha":0.0001},
    {"hidden_layer_sizes": (200, 100), "learning_rate_init": 0.01, "alpha":0.0001},
    {"hidden_layer_sizes": (200, 100), "learning_rate_init": 0.001, "alpha":0.001},
    {"hidden_layer_sizes": (300, 200, 100), "learning_rate_init": 0.001, "alpha":0.001},
]

# Her model için döngü
for idx, parametreler in enumerate(modeller, 1):
    print(f"\n--- MLP Model {idx}: Katmanlar={parametreler['hidden_layer_sizes']}, learning_rate={parametreler['learning_rate_init']}, alpha={parametreler['alpha']} ---")
    
    egitim_basla = time.time()
    
    mlp_model = MLPClassifier(
        hidden_layer_sizes=parametreler["hidden_layer_sizes"],
        activation='relu',
        max_iter=300,
        learning_rate_init=parametreler["learning_rate_init"],
        alpha=parametreler["alpha"],
        random_state=42
    )
    mlp_model.fit(X_train, y_train)
    
   
    egitim_bitis = time.time()
    
    
    test_basla = time.time()
    y_tahmin = mlp_model.predict(X_test)
    test_bitis = time.time()
    
    # Doğruluk hesapla
    dogruluk = accuracy_score(y_test, y_tahmin)
    print("Doğruluk:", dogruluk)
    print(classification_report(y_test, y_tahmin))
    
    # Süreleri yazdır
    egitim_sure = egitim_bitis - egitim_basla
    test_sure = test_bitis - test_basla
    print(f"Eğitim süresi: {egitim_sure:.2f} saniye")
    print(f"Test süresi: {test_sure:.2f} saniye")



--- MLP Model 1: Katmanlar=(100,), learning_rate=0.001, alpha=0.0001 ---
Doğruluk: 0.5978947368421053
              precision    recall  f1-score   support

           0       0.41      0.41      0.41       263
           1       0.75      0.78      0.76       748
           2       0.59      0.56      0.58       805
           3       0.54      0.55      0.55       787
           4       0.53      0.52      0.53       247

    accuracy                           0.60      2850
   macro avg       0.56      0.56      0.56      2850
weighted avg       0.60      0.60      0.60      2850

Eğitim süresi: 37.33 saniye
Test süresi: 0.01 saniye

--- MLP Model 2: Katmanlar=(200, 100), learning_rate=0.001, alpha=0.0001 ---
Doğruluk: 0.6052631578947368
              precision    recall  f1-score   support

           0       0.41      0.41      0.41       263
           1       0.75      0.80      0.77       748
           2       0.58      0.55      0.56       805
           3       0.57      0.

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import time

# Farklı Random Forest parametrelerini burada deniyorum
modeller = [
    {"n_estimators": 100, "max_depth": None},
    {"n_estimators": 200, "max_depth": None},
    {"n_estimators": 300, "max_depth": None},
    {"n_estimators": 100, "max_depth": 10},
    {"n_estimators": 200, "max_depth": 10},
    {"n_estimators": 300, "max_depth": 10},
]

# Her model için döngü
for idx, parametreler in enumerate(modeller, 1):
    print(f"\n--- Random Forest Model {idx}: n_estimators={parametreler['n_estimators']}, max_depth={parametreler['max_depth']} ---")
    
    egitim_basla = time.time()
    
    rf_model = RandomForestClassifier(
        n_estimators=parametreler["n_estimators"],
        max_depth=parametreler["max_depth"],
        random_state=42
    )
    rf_model.fit(X_train, y_train)
    
   
    egitim_bitis = time.time()
    
  
    test_basla = time.time()
    y_tahmin = rf_model.predict(X_test)
    test_bitis = time.time()
    
   
    dogruluk = accuracy_score(y_test, y_tahmin)
    print("Doğruluk:", dogruluk)
    print(classification_report(y_test, y_tahmin))
    
   
    egitim_sure = egitim_bitis - egitim_basla
    test_sure = test_bitis - test_basla
    print(f"Eğitim süresi: {egitim_sure:.2f} saniye")
    print(f"Test süresi: {test_sure:.2f} saniye")



--- Random Forest Model 1: n_estimators=100, max_depth=None ---
Doğruluk: 0.5371929824561403
              precision    recall  f1-score   support

           0       0.57      0.13      0.21       263
           1       0.64      0.77      0.70       748
           2       0.50      0.59      0.54       805
           3       0.47      0.52      0.49       787
           4       0.61      0.19      0.29       247

    accuracy                           0.54      2850
   macro avg       0.56      0.44      0.45      2850
weighted avg       0.54      0.54      0.51      2850

Eğitim süresi: 30.12 saniye
Test süresi: 0.06 saniye

--- Random Forest Model 2: n_estimators=200, max_depth=None ---
Doğruluk: 0.5456140350877193
              precision    recall  f1-score   support

           0       0.53      0.12      0.19       263
           1       0.65      0.77      0.70       748
           2       0.51      0.61      0.55       805
           3       0.48      0.53      0.50       787

In [11]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
import time

# Farklı XGBoost parametrelerini deniyorum
modeller = [
    {"max_depth":3, "learning_rate":0.1, "n_estimators":100},
    {"max_depth":5, "learning_rate":0.1, "n_estimators":100},
    {"max_depth":3, "learning_rate":0.01, "n_estimators":100},
    {"max_depth":5, "learning_rate":0.01, "n_estimators":100},
    {"max_depth":3, "learning_rate":0.1, "n_estimators":200},
    {"max_depth":5, "learning_rate":0.1, "n_estimators":200},
]

# Her model için döngü
for idx, parametreler in enumerate(modeller, 1):
    print(f"\n--- XGBoost Model {idx}: max_depth={parametreler['max_depth']}, learning_rate={parametreler['learning_rate']}, n_estimators={parametreler['n_estimators']} ---")
    
    
    egitim_basla = time.time()
    
    xgb_model = XGBClassifier(
        eval_metric='mlogloss',
        max_depth=parametreler["max_depth"],
        learning_rate=parametreler["learning_rate"],
        n_estimators=parametreler["n_estimators"],
        random_state=42
    )
    xgb_model.fit(X_train, y_train)
    
    
    egitim_bitis = time.time()
    
    
    test_basla = time.time()
    y_tahmin = xgb_model.predict(X_test)
    test_bitis = time.time()
    
   
    dogruluk = accuracy_score(y_test, y_tahmin)
    print("Doğruluk:", dogruluk)
    print(classification_report(y_test, y_tahmin))
    
   
    egitim_sure = egitim_bitis - egitim_basla
    test_sure = test_bitis - test_basla
    print(f"Eğitim süresi: {egitim_sure:.2f} saniye")
    print(f"Test süresi: {test_sure:.2f} saniye")



--- XGBoost Model 1: max_depth=3, learning_rate=0.1, n_estimators=100 ---
Doğruluk: 0.5870175438596491
              precision    recall  f1-score   support

           0       0.50      0.27      0.35       263
           1       0.70      0.76      0.73       748
           2       0.56      0.60      0.58       805
           3       0.52      0.57      0.54       787
           4       0.61      0.42      0.50       247

    accuracy                           0.59      2850
   macro avg       0.58      0.52      0.54      2850
weighted avg       0.58      0.59      0.58      2850

Eğitim süresi: 10.33 saniye
Test süresi: 0.00 saniye

--- XGBoost Model 2: max_depth=5, learning_rate=0.1, n_estimators=100 ---
Doğruluk: 0.5989473684210527
              precision    recall  f1-score   support

           0       0.49      0.25      0.33       263
           1       0.73      0.79      0.76       748
           2       0.56      0.62      0.59       805
           3       0.53      0.56